In [1]:
# Instalar SDK java 8

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Descargar Spark

!wget -q https://archive.apache.org/dist/spark/spark-3.3.4/spark-3.3.4-bin-hadoop3.tgz

# Descomprimir la version de Spark

!tar xf spark-3.3.4-bin-hadoop3.tgz

# Establecer las variables de entorno

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.4-bin-hadoop3"

# Descargar findspark

!pip install -q findspark


In [2]:
!pip install python-dotenv

In [7]:
from pyspark.sql import SparkSession

In [3]:
import findspark

findspark.init()


Extraer credenciales del archivo

In [4]:
from dotenv import dotenv_values
config = dotenv_values("/content/.env.txt")

In [5]:
accessKeyId  = config.get("ACCESS_KEY")
secretAccessKey = config.get("SECRET_ACCESS_KEY")

Crear session de spark con las configuraciones necesarias

In [8]:
spark = (SparkSession
         .builder
         .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,com.amazonaws:aws-java-sdk-bundle:1.11.469")
         .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
         .getOrCreate()
         )

Estraer las credenciales del diccionario

Establecer las configuraciones de hadoop necesarias

In [9]:

sc = spark.sparkContext

sc._jsc.hadoopConfiguration().set('fs.s3a.access.key', accessKeyId)
sc._jsc.hadoopConfiguration().set('fs.s3a.secret.key', secretAccessKey)
sc._jsc.hadoopConfiguration().set('fs.s3a.path.style.access', 'true')
sc._jsc.hadoopConfiguration().set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
sc._jsc.hadoopConfiguration().set('fs.s3a.endpoint', 's3.amazonaws.com')


leer archivo parquet almacenado en S3

In [10]:
print(spark.version)  # versión de Spark
print(sc._jvm.org.apache.hadoop.util.VersionInfo.getVersion())  # versión de Hadoop

3.3.4
3.3.2


In [14]:
df = spark.read.parquet('s3a://mi-bucket-pruebas-abel/datos.parquet')

In [16]:
df.show()

+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|       publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13 17:13:01|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           False| 

Escribir en S3

In [17]:
df.write.parquet('s3a://mi-bucket-pruebas-abel/salida')